# 入门篇

## 1.概念

### 1.1.Python方向

讲了这么多的东西，那Python能干啥呢？这个是**我眼中的Python**：

**Python方向**：
1. 早期方向
    - **Web全栈**
2. 擅长专栏
    - **爬虫系列**
    - **数据分析**
    - **人工智能**
    - **`物联网系`**（`lot`万物互联）
    - **自动化运维**（`安全`与`测试`）
3. 其他系列
    - 游戏开发（最近很火）

如果想在Web、爬虫、物联网、游戏等等方向，网络这块是必需条件，So ==> 不要不急，咱们继续学习～

### 1.2.




## 2.UDP

### 2.1.UDP发送消息

#### 引入案例

看个UDP的简单案例：
```py
import socket

def main():

    # AF_INET ==> IPV4；SOCK_STREAM ==> 类型是TCP，stream 流
    # SOCK_DGRAM ==> 类型是UDP，dgram 数据报、数据报套接字
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as udp_sock:
        udp_sock.sendto("大兄弟，你好啊".encode("utf-8"), ("192.168.36.235", 8080))
    print("over")

if __name__ == '__main__':
    main()
```
接收到的消息：**这时候端口是随机的**
![2.UDP接收消息](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030161939823-691450792.png)

看起来代码还挺麻烦，我稍微分析下你就知道对比其他语言真的太简单了：

标识：
1. `AF_INET` ==> `IPV4`
2. `SOCK_DGRAM` ==> 类型是`UDP`
3. `SOCK_STREAM` ==> 类型是`TCP`

**代码三步走**：
1. 创建 `udp_sock=socket.socket(socket.AF_INET, socket.SOCK_DGRAM)`
2. 发送 `udp_sock.sendto(Bytes内容，(IP,Port))`
3. 关闭 `udp_sock.close()`

#### 端口绑定

借助`调试工具`(<a href="https://files.cnblogs.com/files/dunitian/NetTool.tar.gz" target="_blank">点我下载</a>)可以知道：上面程序每次运行，**端口**都**不固定**
![2.UDP随机端口.png](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030160106572-1670415822.png)

那怎么使用固定端口呢？==> `udp_socket.bind(('', 5400))`
```py
import socket

def main():
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as udp_socket:
        # 绑定固定端口
        udp_socket.bind(('', 5400))
        # 发送消息
        udp_socket.sendto("小明，你知道小张的生日吗？\n".encode("utf-8"),
                          ("192.168.36.235", 8080))
    print("over")

if __name__ == '__main__':
    main()
```
消息图示：`nc -ul 8080`（`nc -l`是监听TCP）
![2.nc监听UDP.png](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030161609064-948308184.png)

调试工具：
![2.UDP绑定端口.png](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030161114687-141010359.png)

### 2.2.UDP接收消息

先看一个简单版本的：`udp_socket.recvfrom(1024)`
```py
from socket import socket, AF_INET, SOCK_DGRAM

def main():
    with socket(AF_INET, SOCK_DGRAM) as udp_socket:
        # 绑定端口
        udp_socket.bind(('', 5400))
        while True:
            # 发送消息
            udp_socket.sendto("你可以给我离线留言了\n".encode("utf-8"),
                              ("192.168.36.235", 8080))
            # 接收消息（data,(ip,port)）
            data, info = udp_socket.recvfrom(1024)
            print(f"[来自{info[0]}:{info[1]}的消息]：\n{data.decode('utf-8')}")

if __name__ == '__main__':
    main()
```
图示：**`接收消息（data,(ip,port)）`**
![2.udp_recv.gif](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030172716634-871024783.gif)

---

### 题外话（Nmap）

其实如果你使用`Nmap`来扫描的话并不能发现`nc`打开的`UDP`端口：
![2.nmap的UDP扫描.gif](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030172711902-1875073285.gif)

稍微解释一下：**扫描其实就是发了几个空消息过去**
1. `-sU`代表扫描UDP，`-sT`代表扫描TCP
2. `-Pn` 这个主要是针对有些服务器禁用ping的处理（ping不通也尝试）
3. `-p` 指定端口号，如果是所有端口可以使用`-p-`
4. `sudo`是因为在`Ubuntu`下没权限，`kali`下可以直接使用`nmap`

可能有人对`nc`输出的*`你可以给离线留意了`*有疑惑，其实就是在给5400端口发空消息的时候～`True循环了两次`

来张对比图：
![2.nc找不到.gif](https://img2018.cnblogs.com/blog/658978/201810/658978-20181031093925016-1829175408.gif)

**扫描TCP和UDP端口**：`sudo nmap -sTU 192.168.36.235 -Pn`

**课后扩展**：
```
NC命令扩展：https://www.cnblogs.com/nmap/p/6148306.html

Nmap基础：https://www.cnblogs.com/dunitian/p/5074784.html
```

#### 收放自如

如果还是用True循环来实现：
```py
from socket import socket, AF_INET, SOCK_DGRAM

def main():
    with socket(AF_INET, SOCK_DGRAM) as udp_socket:
        # 绑定端口
        udp_socket.bind(('', 5400))
        while True:
            msg = input("请输入发送的内容：")
            if msg == "dotnetcrazy":
                break
            else:
                udp_socket.sendto(
                    msg.encode("utf-8"), ("192.168.36.235", 8080))

            data, info = udp_socket.recvfrom(1024)
            print(f"[来自{info[0]}:{info[1]}的消息]：\n{data.decode('utf-8')}")

if __name__ == '__main__':
    main()
```
你会发现，消息不能轮流发送，只能等对方方式后再发，虽然有处理方式，但太麻烦，这时候就可以使用我们之前说的**多线程来改写**一下了：
```py
from socket import socket, AF_INET, SOCK_DGRAM
from multiprocessing.dummy import Pool as ThreadPool

def send_msg(udp_socket):
    while True:
        msg = input("输入需要发送的消息：\n")
        udp_socket.sendto(msg.encode("utf-8"), ("192.168.36.235", 8080))

def recv_msg(udp_socket):
    while True:
        data, info = udp_socket.recvfrom(1024)
        print(f"[来自{info[0]}:{info[1]}的消息]：\n{data.decode('utf-8')}")

def main():
    # 创建一个Socket
    with socket(AF_INET, SOCK_DGRAM) as udp_socket:
        # 绑定端口
        udp_socket.bind(('', 5400))

        # 创建一个线程池
        pool = ThreadPool()

        # 接收消息
        pool.apply_async(recv_msg, args=(udp_socket, ))

        # 发送消息
        pool.apply_async(send_msg, args=(udp_socket, ))

        pool.close()  # 不再添加任务
        pool.join()  # 等待线程池执行完毕
    print("over")

if __name__ == '__main__':
    main()
```
输出：（就一个注意点～`socket在pool之后关闭`）
![2.收放自如.gif](https://img2018.cnblogs.com/blog/658978/201810/658978-20181031084637702-678648461.gif)

---

### 2.3.手写UDP网络调试工具

调试工具功能比较简单，我们手写一个`UDP`版的：
```py
from socket import socket, AF_INET, SOCK_DGRAM
from multiprocessing.dummy import Pool as ThreadPool

def get_port(msg):
    """获取用户输入的端口号"""
    while True:
        port = input(msg)
        try:
            port = int(port)
        except Exception as ex:
            print(ex)
        else:
            return port  # 没有错误就退出死循环

def recv_msg(udp_socket):
    """接收消息"""
    while True:
        data, info = udp_socket.recvfrom(1024)
        print(f"[来自{info[0]}:{info[1]}的消息]：\n{data.decode('utf-8')}")

def send_msg(udp_socket):
    """发送消息"""
    ip = input("请输入对方IP：")
    port = get_port("请输入对方端口号：")
    while True:
        msg = input("请输入发送的消息：\n")
        udp_socket.sendto(msg.encode("utf-8"), (ip, port))

def main():
    with socket(AF_INET, SOCK_DGRAM) as udp_socket:
        # 绑定端口
        udp_socket.bind(('', get_port("请输网络助手的端口号：")))
        # 创建一个线程池
        pool = ThreadPool()
        # 接收消息
        pool.apply_async(recv_msg, args=(udp_socket, ))
        # 发送消息
        pool.apply_async(send_msg, args=(udp_socket, ))

        pool.close()
        pool.join()

if __name__ == '__main__':
    main()

```

CentOS`IP`和`Port`(`192.168.36.123:5400`)
![2.UDP网络助手.png](https://img2018.cnblogs.com/blog/658978/201810/658978-20181031130707093-968035575.png)

演示：(**多PC演示**)
![2.UDPTool.gif](https://img2018.cnblogs.com/blog/658978/201810/658978-20181031131353347-509340602.gif)

简单说下本机IP的绑定：

Net里面习惯使用`localhost`，很多人不知道到底是啥，其实你打开`host`文件就可以看到 ==> `127.0.0.1`被重定向为`localhost`，在Linux里面也是这样的，每个PC对应的都是`lo`回环地址：
![2.lo.png](https://img2018.cnblogs.com/blog/658978/201810/658978-20181031132010394-1833915775.png)

**本机通信时，对方ip就可以使用`127.0.0.1`了，当然了绑定本机ip的时候也可以使用`127.0.0.1`**

## 3.TCP

用过下载软件的可能遇到过一种`‘Bug’` ==> 很多人为了防止自己本地文件纳入共享大军，一般都是直接把网络上传给禁了，然后发现文件经常出问题？

其实这个就是`TCP`的一个应用，文件一般都很大，所以进行分割后批量下载，那少量的网络上传其实是为了校验一下文件 ==> 正确做法是限制上传速度而不是禁止（学生时代那会还经常蛋疼这个问题，现在想想还挺好玩的`O(∩_∩)O哈哈~`）

### 3.1.TCP发送消息

### 3.2.TCP接收消息

# 加强篇

# 深入篇